<div style="background-color:#f0f8ff; border-radius:8px; padding:12px; text-align:center;">

# **PRACTICA 2: Árboles y Ensembles**

</div>

*Aprendizaje Automático*

---

**Grupo:** G-7312  
**Número de pareja:** 01  
**Miembros:**  
- Leire Bernárdez Vázquez  
- Carmen Reiné Rueda

---

### **Importaciones**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_covtype
from sklearn.tree import DecisionTreeClassifier

---

### **1. Clasificación mediante árboles de clasificación**

1. Cargue los datos del problema

In [ ]:
# Load the Covertype dataset
cov_type = fetch_covtype()
# Separate the features and the target variable
X = cov_type.data
y = cov_type.target

2. Separe los datos en training (50 %) y test (50 %)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=42, stratify=y
)

No hace falta escalarlo, lo dice la teacher (los árboles se tragan todo lo que les des)

3. Utilice la clase DecisionTreeClassifier de Sklearn para resolver el problema
y dé un resultado de test usando la m ́etrica que considere oportuna

In [13]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)
y_pred = decision_tree.predict(X_test)
porcentaje = accuracy_score(y_test, y_pred)
print("Accuracy:", porcentaje*100, "%")

Accuracy: 92.34611333328743 %


4. ¿Había missing values en nuestro problema? ¿Qué efecto tendría en los árboles no completar los missing values ?

In [14]:
np.isnan(X_train).sum()==0

True

Si existieran y no se imputaran, DecisionTreeClassifier daría error al entrenar, porque los árboles no soportan missing values; requieren que todos los datos estén completos para funcionar. Para solucionar este problema, se podrían imputar con media, mediana o un valor concreto.

5. ¿Con qué parémetro podría controlarse la profundidad del ́arbol? ¿Qué ocurre si un ́arbol se inicializa sin dar un valor concreto a ese parámetro? ¿Crees que sería una buena práctica no darle ningún valor?

a) La profundidad máxima del árbol se controla con el parámetro max_depth directamente, que por defecto es max_depth=None. Sin embargo, también se puede controlar indirectamente con min_samples_split, que no divide un nodo si tiene menos muestras que las requeridas, y con min_samples_leaf, que obliga a que cada hoja tenga un mínimo de muestras para evitar que existan ramas muy profundas con pocas muestras. Por lo tanto, con estos valores se reduce la profundidad máxima alcanzada.

b) Crecería tanto como datos se le pasaran sin ningún tipo de límite.

c) No, debido a que sin un límite establecido, los árboles grandes consumen más memoria, tardan más en entrenar y tienden a sobreajustarse más fácilmente. Con lo cual, sería importante definir un max_depth razonable teniendo en cuenta los datos.

6. ¿Son sensibles los ́arboles al desequilibrio entre clases?

7. ¿Existe desequilibrio entre las clases de nuestro problema? ¿Con qué parámetro de la clase DecisionTreeClassifier podrías corregirlo?

8. ¿Cuál es la diferencia entre los parámetros min samples split y min samples leaf ?

9. ¿Para qué sirve el parémetro criterion, qué valores puede tomar y qué significa o representa cada uno de esos valores?

10. El ́arbol entrenado, ¿realiza splits utilizando todas las variables disponibles?

11. ¿Se le ocurre alguna forma de crear un selector de atributos a partir de un árbol de clasificación?

---

### **2. Regresión mediante árboles de regresión**

---

### **3. Ensembles**